<!-- dom:TITLE: Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version -->
# Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version
<!-- dom:AUTHOR: Øyvind Ryan -->
<!-- Author: --> **Øyvind Ryan**

Date: **Oct 24, 2018**

<!-- Externaldocuments: applinalg -->
<!-- Mapping from exercise labels to numbers: label2numbers = {'exercise:verify_statement_filters': '7.14', 'exercise:5397many': '7.8', 'exercise:ex5kernel': '7.9', 'exercise:verify_near_perfect': '7.15', 'exercise:080301': '7.13'} -->

# The polyphase representation of filter bank transforms













<!-- --- begin exercise --- -->

## Exercise 7.8: Lifting-based implementations of the Spline 5/3 and CDF 9/7 wavelets
<div id="exercise:5397many"></div>
<!-- keywords = wavlifting; student -->

Let us use the different lifting factorizations obtained in this chapter to implement the corresponding wavelet kernels.
Your functions should call the functions from Exercise 4.24 and 4.29. 
You will need equations (7.8)-(7.11) here, in order to complete the kernels.

In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), 'python'))

In [2]:
%matplotlib inline

from numpy import *
from forward_compress_reverse import *
from mp3funcs import *
import matplotlib.pyplot as plt

**a)**
Write functions

        dwt_kernel_53(x, bd_mode)
        idwt_kernel_53(x, bd_mode)


which implement the DWT and IDWT kernels for the Spline 5/3 wavelet.
Use the lifting factorization obtained in Example 7.2.3.


<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look like this:

In [3]:
def dwt_kernel_53(x, bd_mode):
    x[0::2] *= 2
    x[1::2] *= 0.5
    lifting_odd_symm(-0.125, x, bd_mode)
    lifting_even_symm(1, x, bd_mode)

In [4]:
def idwt_kernel_53(x, bd_mode):
    lifting_even_symm(-1, x, bd_mode)
    lifting_odd_symm(0.125, x, bd_mode)     
    x[0::2] *= 0.5
    x[1::2] *= 2

<!-- --- end solution of exercise --- -->


**b)**
Write functions

        dwt_kernel_97(x, bd_mode)
        idwt_kernel_97(x, bd_mode)


which implement the DWT and IDWT kernels for the CDF 9/7 wavelet. 
Use the lifting factorization obtained in Example 7.2.4.


<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look like this:

In [5]:
def dwt_kernel_97(x, bd_mode):
    lambda1=-0.586134342059950
    lambda2=-0.668067171029734
    lambda3=0.070018009414994
    lambda4=1.200171016244178
    alpha=-1.149604398860250
    beta=-0.869864451624777
    
    x[0::2] *= alpha
    x[1::2] *= beta
    lifting_odd_symm(-lambda4, x, bd_mode)
    lifting_even_symm(-lambda3, x, bd_mode)
    lifting_odd_symm(-lambda2, x, bd_mode)
    lifting_even_symm(-lambda1, x, bd_mode)

In [6]:
def idwt_kernel_97(x, bd_mode):
    lambda1=-0.586134342059950
    lambda2=-0.668067171029734
    lambda3=0.070018009414994
    lambda4=1.200171016244178
    alpha=-1.149604398860250
    beta=-0.869864451624777
        
    lifting_even_symm(lambda1, x, bd_mode)
    lifting_odd_symm(lambda2, x, bd_mode)   
    lifting_even_symm(lambda3, x, bd_mode)
    lifting_odd_symm(lambda4, x, bd_mode)      
    x[0::2] /= alpha
    x[1::2] /= beta

<!-- --- end solution of exercise --- -->

**c)**
In Chapter 4, 
we listened to the low-resolution approximations and detail components in sound for three different wavelets. 
Repeat these experiments with the Spline 5/3 and the CDF 9/7 wavelet, using the new kernels you implemented
in a) and b).


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used for listening to the low-resolution approximations for a given value of $m$.

In [7]:
m = 1
x, fs = forw_comp_rev_dwt1(m, 'spline53')
play(x, fs)

In [8]:
m = 1
x, fs = forw_comp_rev_dwt1(m, 'cdf97')
play(x, fs)

<!-- --- end solution of exercise --- -->

**d)**
Plot all scaling functions and mother wavelets for the Spline 5/3
and the CDF 9/7 wavelets, using the cascade algorithm and the kernels you have implemented.


<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look as follows.

In [9]:
m=10
cascade_alg(m, -4, 4, 'spline53', True, False)
cascade_alg(m, -4, 4, 'spline53', False, False)

cascade_alg(m, -4, 4, 'cdf97', True, False)
cascade_alg(m, -4, 4, 'cdf97', False, False)

In the plot for the CDF 9/7 wavelet, it is seen that the functions and their dual counterparts are close to being equal. 
This reflects the fact that this wavelet is close to being orthogonal.

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 7.9: Lifting-based implementation of orthonormal wavelets
<div id="exercise:ex5kernel"></div>
<!-- keywords = wavlifting -->



**a)**
Write functions

        lifting_even(lambda1, lambda2, x, bd_mode)
        lifting_odd(lambda1, lambda2, x, bd_mode)


which apply the elementary
lifting matrices (7.14) to $\boldsymbol{x}$. Assume that $N$ is even.


<!-- --- begin solution of exercise --- -->
**Solution.**
The library versions of these functions are as follows.

        def lifting_even(lmbda1, lmbda2, x, bd_mode):
            if mod(len(x), 2)!=0:
                raise AssertionError()
            x[0] += lmbda1*x[1] + lmbda2*x[-1]
            x[2:-1:2] += lmbda1*x[3::2] + lmbda2*x[1:-2:2]


        def lifting_odd(lmbda1, lmbda2, x, bd_mode):
            if mod(len(x), 2)!=0:
                raise AssertionError()
            x[1:-2:2] += lmbda1*x[2:-1:2] + lmbda2*x[0:-3:2]
            x[-1] += lmbda1*x[0] + lmbda2*x[-2]


<!-- --- end solution of exercise --- -->


**b)**
Write functions

        dwt_kernel_ortho(x, bd_mode, wav_props)
        idwt_kernel_ortho(x, bd_mode, wav_props)


which apply the DWT and IDWT kernels for orthonormal wavelets, using the functions `lifting_even` and `lifting_odd`. 
Assume that you can access lifting steps so that the lifting factorization (7.7) holds through the object `wav_props` by means of writing 
`wav_props.lambdas`, `wav_props.alpha`, and `wav_props.beta`, and that `wav_props.last_even` indicates whether the last lifting step is even.
`wav_props.lambdas` is an $n\times 2$-matrix so that the filter coefficients 
$\{\underline{\lambda_1},\lambda_2\}$ or $\{\lambda_1,\underline{\lambda_2}\}$ in the $i$'th
lifting step are found in row $i$. 

You can now define standard DWT and IDWT kernels in the following way, once the `wav_props` object has been computed for a given $N$:

        dwt_kernel = lambda x, bd_mode: dwt_kernel_ortho(x, bd_mode, wav_props)


<!-- --- begin solution of exercise --- -->
**Solution.**
The library versions of these functions are as follows, and again support more general boundary modes:

        def _dwt_kernel_ortho(x, bd_mode, dual_wav_props):
            # TODO: Add boundary handling
            x[0::2] /= dual_wav_props.alpha
            x[1::2] /= dual_wav_props.beta
            iseven = not dual_wav_props.last_even
            for stepnr in range(dual_wav_props.lambdas.shape[0] - 1, -1, -1):
                if iseven:
                    lifting_even( dual_wav_props.lambdas[stepnr, 1], \
                                  dual_wav_props.lambdas[stepnr, 0], \
                                  x, bd_mode)
                else:    
                    lifting_odd(  dual_wav_props.lambdas[stepnr, 1], \
                                  dual_wav_props.lambdas[stepnr, 0], \
                                  x, bd_mode)
                iseven = not iseven
            # TODO: Add boundary handling


        def _idwt_kernel_ortho(x, bd_mode, wav_props):
            # TODO: Add boundary handling
            iseven = ( mod(wav_props.lambdas.shape[0], 2) == wav_props.last_even )
            for stepnr in range(wav_props.lambdas.shape[0]):
                if iseven:
                    lifting_even( wav_props.lambdas[stepnr, 0], \
                                  wav_props.lambdas[stepnr, 1], \
                                  x, bd_mode)
                else:    
                    lifting_odd(  wav_props.lambdas[stepnr, 0], \
                                  wav_props.lambdas[stepnr, 1], \
                                  x, bd_mode)
                iseven = not iseven
            x[0::2] /= wav_props.alpha
            x[1::2] /= wav_props.beta
            # TODO: Add boundary handling


<!-- --- end solution of exercise --- -->

**c)**
Listen to the low-resolution approximations and detail components in sound for orthonormal wavelets for $N=1,2,3,4$.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used for listening to the low-resolution approximations for a given value of $m$.

In [10]:
x, fs = forw_comp_rev_dwt1(m, 'db2')
play(x, fs)

In [11]:
x, fs = forw_comp_rev_dwt1(m, 'db3')
play(x, fs)

In [12]:
x, fs = forw_comp_rev_dwt1(m, 'db4')
play(x, fs)

<!-- --- end solution of exercise --- -->

**d)**
Plot all scaling functions and mother wavelets for the orthonormal
wavelets for $N=1,2,3,4$, using the cascade algorithm. 
Since the wavelets are orthonormal, we should have that $\phi=\tilde{\phi}$, and $\psi=\tilde{\psi}$. 
In other words, you should see that the bottom plots
equal the upper plots.


<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look as follows.

In [13]:
m=10
cascade_alg(m, -4, 4, 'db2', True, False)
cascade_alg(m, -4, 4, 'db2', False, False)

cascade_alg(m, -4, 4, 'db3', True, False)
cascade_alg(m, -4, 4, 'db3', False, False)

cascade_alg(m, -4, 4, 'db4', True, False)
cascade_alg(m, -4, 4, 'db4', False, False)

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 7.13: Run forward and reverse transform
<div id="exercise:080301"></div>
<!-- keywords = wavlifting; student -->

Run the forward and then the reverse transform from Exercise 5.30 on the vector $(1,2,3,\ldots,8192)$.
Verify that there seems to be a delay on $481$ elements, as promised in Section 7.3.4. 
Do you get the exact same result back?


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used:

In [14]:
x = arange(1,8193)
mp3_reverse_fbt(mp3_forward_fbt(x))
plt.plot(x)

There are some small errors from the original vector in the resulting vector, when one compensates for the delay of $481$ elements.

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 7.14: Verify statement of filters
<div id="exercise:verify_statement_filters"></div>
<!-- keywords = wavlifting; student -->

Use your computer to verify the following symmetries for the prototype filters:

$$
C_i = \begin{cases} -C_{512-i} & i\neq 64,128,\ldots,448 \\ C_{512-i} & i=64,128,\ldots,448. \end{cases}
$$

Explain also that this implies that $h_i=h_{512-i}$ for $i=1,\ldots,511$. Recall that the modified version had the symmetry $h_i=h_{511-i}$.  
When the filters $V^{(m)}$ are defined as in this section, explain why (7.25) should be replaced by

$$
V^{(64-k)} =-E_{14}(V^{(k)})^T
$$

in the MP3 standard


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used

In [15]:
ctable = mp3_ctable()
ctable = ctable[1::]
ctable[63:255:64] = -ctable[63:255:64]
res = ctable + ctable[-1::-1]
res[255]=0
print(abs(res).max())

Note that the middle element of the table is set to $0$ in this verification. The reason is that, for $i=256$, there is nothing to check. 
So, since the verification works by producing zero when there is symmetry, the element corresponding to $i=256$ is set to zero. 
Also, every $64$'th element in one half of the table changed sign, in order to establish symmetry for the corresponding entries, 
rather than anti-symmetry as for the other entries.

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 7.15: Verify near perfect reconstruction in the MP3 standard
<div id="exercise:verify_near_perfect"></div>
<!-- keywords = wavlifting -->

In Section 7.3.3 we saw that the polyphase representation of $GH$ is

$$
16\begin{pmatrix} S_0 & \cdots & \boldsymbol{0} \\ \vdots & \ddots & \vdots \\ \boldsymbol{0} & \cdots & S_{31} \end{pmatrix}E_{480},
$$

where $S_i = (V^{(16+i)})^TV^{(16+i)} + (V^{(48+i)})^TV^{(48+i)}$. As explained in Section 7.3.4, the MP3 standard chooses the prototype filter so that the $S_i$ are equal
(so that the block matrix above is a filter), but only near perfect reconstruction is possible.


**a)**
With $S_i=\{...,t_{-1},\underline{t_0},t_1,...\}$, explain why the (magnitude of the) frequency response of (the filter) $GH$ equals

$$
16\left|\sum_{k} t_k e^{-iMk\omega}\right|.
$$

<!-- --- begin solution of exercise --- -->
**Solution.**
Disregarding the delay $E_{480}$, which does not affect the absolute value of the frequency response, 
the filter coefficients of $GH$ come at indices $0$, $M$, $2M$, and so on, and are $16t_0$, $16t_1$, $16t_2$, and so on. This gives

$$
\begin{align*}
|\lambda_{GH}(\omega)| &= 16\left|\sum_k t_ke^{-iMk\omega}\right|.
\end{align*}
$$

<!-- --- end solution of exercise --- -->

**b)**
Use the values in the table $C$ to find the filter coefficients $t_k$ for

$$
S_1= (V^{(17)})^TV^{(17)} + (V^{(49)})^TV^{(49)}.
$$

(use (7.19), even though this filter bank does not give perfect reconstruction), and plot this frequency response. 
If you get an almost flat frequency response, you have verified near perfect reconstruction in the MP3 standard. It is important that you scale the vertical axis in order to see that the frequency response is close to constant.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used.

In [16]:
C = mp3_ctable()

V1 = zeros(15)
V1[::2]=C[17::64]

V2 = zeros(15)
V2[::2]=C[50::64]
diagentry = convolve(V1[(-1)::(-1)], V1)+convolve(V2[(-1)::(-1)], V2)

S = zeros(32*len(diagentry))
S[::32] = diagentry; # upsampled
S = concatenate( (S, zeros(16384) ), axis=0)
N = len(S)
n = arange(0,N)

plt.plot(2*pi*n/N, abs(fft.fft(S)), 'k-')
plt.axis([0, 2*pi, 0, 1.5/512])

<!-- --- end solution of exercise --- -->

**c)**
Verify that the frequency response of $S_1$ in b) should be close to $1/(16\cdot 32) = 1/512$. 
The factor $32$ has to do with the relative scaling of the $C$ and $D$ tables.


<!-- --- begin solution of exercise --- -->
**Solution.**
Since the values in the table D are $32$ times those in the table C, we will instead have

$$
\lambda_S(\omega)=(16\cdot 32)\left|\sum_{k} t_k e^{-iMk\omega}\right| = 512\left|\sum_{k} t_k e^{-iMk\omega}\right|
$$

(recall that $S_i$ represents values from the table C). For this to be close to one we must clearly have $\left|\sum_{k} t_k e^{-iMk\omega}\right|\approx 1/512$.

<!-- --- end solution of exercise --- -->



<!-- --- end exercise --- -->